In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='ticks')


import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression

C:\Users\Ryan\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
cred = pd.read_csv('../../creditcard.csv')

In [3]:
cred.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
cred.shape

(284807, 31)

In [5]:
np.bincount(cred.Class.ravel())

array([284315,    492], dtype=int64)

In [6]:
cred.groupby('Class').size()

Class
0    284315
1       492
dtype: int64

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
cred['Amount_norm'] = StandardScaler().fit_transform(cred['Amount'].values.reshape(-1,1))

C:\Users\Ryan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [9]:
X = cred.drop(['Time', 'Class', 'Amount'], 1)

In [10]:
X.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount_norm'],
      dtype='object')

In [11]:
l = list(range(0, len(X.columns))) # rename columns to numbers
X = X.rename(columns=dict(zip(X.columns, l))) 

In [12]:
y = cred.Class.ravel()

In [13]:
from sklearn.model_selection import train_test_split,cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,precision_score,classification_report,f1_score 
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

In [14]:
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler

In [15]:
X_Train, X_test, y_Train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

# test set is set right here (in future, test is validation, and vice versa)
# resample THEN get val set...

In [19]:
# oversample
X_ovrsamp, y_ovrsamp = SMOTE(random_state=0).fit_sample(X_Train, y_Train)
X_ovrsamp1, y_ovrsamp1 = SMOTE(kind='borderline1', random_state=0).fit_sample(X_Train, y_Train)
X_ovrsamp2, y_ovrsamp2 = SMOTE(kind='borderline2', random_state=0).fit_sample(X_Train, y_Train)
X_ovrsampada, y_ovrsampada = ADASYN(random_state=0).fit_sample(X_Train, y_Train)
X_ovrsamprand, y_ovrsamprand = RandomOverSampler(random_state=0).fit_sample(X_Train, y_Train)

In [20]:
smote = [X_ovrsamp, y_ovrsamp]
smote1 = [X_ovrsamp1, y_ovrsamp1]
smote2 = [X_ovrsamp2, y_ovrsamp2]
smoteada = [X_ovrsampada, y_ovrsampada]
smoterand = [X_ovrsamprand, y_ovrsamprand]

In [21]:
# regularization 
c_params = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

In [22]:
lrcv = LogisticRegressionCV(Cs=c_params, random_state=777, scoring='recall')
lrcv_1 = LogisticRegressionCV(Cs=c_params, random_state=777, scoring='recall')
lrcv_2 = LogisticRegressionCV(Cs=c_params, random_state=777, scoring='recall')
lrcv_ada = LogisticRegressionCV(Cs=c_params, random_state=777, scoring='recall')
lrcv_rand = LogisticRegressionCV(Cs=c_params, random_state=777, scoring='recall')

In [23]:
# using smote, for example
X_train, X_val, y_train, y_val = train_test_split(X_ovrsamp, y_ovrsamp, random_state=0, test_size=0.1) # smote

In [24]:
print('Test-set Class balance:' ,np.bincount(y_test),
     '\nValidation-set Class balance:' , np.bincount(y_val),
     '\nTrain-set Class balance:', np.bincount(y_train))

Test-set Class balance: [28426    55] 
Validation-set Class balance: [25640 25538] 
Train-set Class balance: [230249 230351]


In [25]:
def log_reg(lrclf, X_resample, y_resample, X_test, y_test):
    '''train_test_split X_resample, y_resample;
        fit to resulting train set;
        cross_validate - print(accuracy, precision, recall, f1, roc_auc);
        '''
    X_train, X_val, y_train, y_val = train_test_split(X_resample, y_resample, random_state=0, test_size=0.1)

    lrclf.fit(X_train, y_train)  
    
    print('-------------------------------------------')
    print('CV')
    print('-------------------------------------------')    
    scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    scores = cross_validate(lrclf, X_train, y_train, cv=3, scoring=scoring, return_train_score=False)
    
    for metric in scoring:
        metr = 'test_' + metric
        print(metric, scores[metr])
        
    print('-------------------------------------------')
    print('Overall')
    print('-------------------------------------------')
    

    y_ = lrclf.predict(X_val)
    print(confusion_matrix(y_val, y_))
    print('Validation - Recall: %0.4f'% recall_score(y_val,y_), 
          'Precision: %0.4f'% precision_score(y_val,y_),
          'Accuracy: %0.4f'% lrclf.score(X_val,y_val))
    
    y_ = lrclf.predict(X_test)
    print(confusion_matrix(y_test, y_))
    print('Test - Recall: %0.4f'% recall_score(y_test,y_), 
          'Precision: %0.4f'% precision_score(y_test,y_),
          'Accuracy: %0.4f'% lrclf.score(X_test,y_test))
    
    
    
        # hold off on below

#     fpr, tpr, thresholds = roc_curve(y_val, y_)
#     roc_auc = auc(fpr, tpr)
    
#     plt.title('Receiver Operating Characteristic')
#     plt.plot(fpr, tpr, 'b',label='AUC = %0.4f'% roc_auc)
#     plt.legend(loc='lower right')
#     plt.plot([0,1],[0,1],'r--')
#     plt.xlim([-0.1,1.0])
#     plt.ylim([-0.1,1.01])
#     plt.ylabel('True Positive Rate')
#     plt.xlabel('False Positive Rate')
#     plt.show()

In [26]:
log_reg(lrcv, X_ovrsamp, y_ovrsamp, X_test, y_test)

-------------------------------------------
CV
-------------------------------------------
accuracy [0.94298331 0.94356299 0.94317797]
precision [0.97393134 0.97532622 0.97279611]
recall [0.91035893 0.9101766  0.91188154]
f1 [0.94107273 0.94162585 0.94135441]
roc_auc [0.98765004 0.98775369 0.98770225]
-------------------------------------------
Overall
-------------------------------------------
[[24980   660]
 [ 2283 23255]]
Validation - Recall: 0.9106 Precision: 0.9724 Accuracy: 0.9425
[[27723   703]
 [    2    53]]
Test - Recall: 0.9636 Precision: 0.0701 Accuracy: 0.9752


In [27]:
log_reg(lrcv_1, X_ovrsamp1, y_ovrsamp1, X_test, y_test)

-------------------------------------------
CV
-------------------------------------------
accuracy [0.98800266 0.98857582 0.98772243]
precision [0.9885525  0.98977753 0.98878824]
recall [0.9874453  0.98735414 0.98663767]
f1 [0.98799859 0.98856435 0.98771178]
roc_auc [0.99871126 0.99885559 0.99872952]
-------------------------------------------
Overall
-------------------------------------------
[[25358   282]
 [  350 25188]]
Validation - Recall: 0.9863 Precision: 0.9889 Accuracy: 0.9877
[[28145   281]
 [    5    50]]
Test - Recall: 0.9091 Precision: 0.1511 Accuracy: 0.9900


In [28]:
log_reg(lrcv_2, X_ovrsamp2, y_ovrsamp2, X_test, y_test)

-------------------------------------------
CV
-------------------------------------------
accuracy [0.98379512 0.9837885  0.98328036]
precision [0.98377892 0.98501731 0.98402714]
recall [0.98380456 0.98251466 0.9825014 ]
f1 [0.98379174 0.98376439 0.98326368]
roc_auc [0.99797792 0.99810702 0.99801365]
-------------------------------------------
Overall
-------------------------------------------
[[25126   413]
 [  462 25177]]
Validation - Recall: 0.9820 Precision: 0.9839 Accuracy: 0.9829
[[28001   425]
 [    4    51]]
Test - Recall: 0.9273 Precision: 0.1071 Accuracy: 0.9849


In [29]:
log_reg(lrcv_ada, X_ovrsampada, y_ovrsampada, X_test, y_test)

-------------------------------------------
CV
-------------------------------------------
accuracy [0.87582636 0.87592976 0.87463933]
precision [0.89496598 0.89462169 0.89072308]
recall [0.85158408 0.85223542 0.85404427]
f1 [0.87273625 0.87291432 0.87199814]
roc_auc [0.95420418 0.95478347 0.95364578]
-------------------------------------------
Overall
-------------------------------------------
[[22984  2594]
 [ 3665 21935]]
Validation - Recall: 0.8568 Precision: 0.8942 Accuracy: 0.8777
[[25583  2843]
 [    0    55]]
Test - Recall: 1.0000 Precision: 0.0190 Accuracy: 0.9002


In [30]:
# using ADASYN oversampling results in the best recall_score
# however, not true for validation set
## why would a model with a lower score in the validation set be selected?? 
### regardless if it did result in the better recall_score for the test set
### the real-world test set would be the actual implementation of the model

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [32]:
dt_g = DecisionTreeClassifier(max_depth=5, random_state=0, criterion='gini')
dt_e = DecisionTreeClassifier(max_depth=5, random_state=0, criterion='entropy')

rfc_g = RandomForestClassifier(max_depth=5, random_state=0, criterion='gini')
rfc_e = RandomForestClassifier(max_depth=5, random_state=0, criterion='entropy')

In [33]:
def trees_and_forests(clf, ovr_samp):
    
    X_train, X_val, y_train, y_val = train_test_split(ovr_samp[0], ovr_samp[1], random_state=0, test_size=0.1)
    
    clf.fit(X_train, y_train)
 
    y_v = clf.predict(X_val)
    y_t = clf.predict(X_test) 
    print('Original features\n')
    print(confusion_matrix(y_val, y_v), 
      'Validation: score=%0.4f' % clf.score(X_val, y_val), '  recall=%0.4f' % recall_score(y_val, y_v), '  precision=%0.4f\n\n' % precision_score(y_val, y_v),
      confusion_matrix(y_test, y_t), 
      'Test: score=%0.4f' % clf.score(X_test, y_test), '  recall=%0.4f' % recall_score(y_test, y_t), '  precision=%0.4f' % precision_score(y_test, y_t),
     )
    
    clf_coef = pd.DataFrame(clf.feature_importances_)
    clf_feats = clf_coef[clf_coef[0]>0.01].sort_values(0, ascending=False)
    clf_index = clf_feats.index.values
    
    
    print('\nThe most important features: ', clf_index)
    print('\n\nImportant features\n')
    
    # train, val, test set with only key features
    X_train_key_feat = pd.DataFrame(X_train)[clf_index]
    X_val_key_feat  = pd.DataFrame(X_val)[clf_index]
    X_test_key_feat = pd.DataFrame(X_test)[clf_index]
    
    clf.fit(X_train_key_feat, y_train)
    
    y_kf_v = clf.predict(X_val_key_feat)
    y_kf_t = clf.predict(X_test_key_feat)
    
    print(confusion_matrix(y_val, y_kf_v), 
      'Validation: score=%0.4f' % clf.score(X_val_key_feat, y_val), '  recall=%0.4f' % recall_score(y_val, y_kf_v), '  precision=%0.4f\n\n' % precision_score(y_val, y_kf_v),
      confusion_matrix(y_test, y_kf_t), 
      'Test: score=%0.4f' % clf.score(X_test_key_feat, y_test), '  recall=%0.4f' % recall_score(y_test, y_kf_t), '  precision=%0.4f' % precision_score(y_test, y_kf_t),
     )  

    return clf_index # maybe return other stuff as well

In [34]:
list_dtg0 = trees_and_forests(dt_g, smote);

Original features

[[24349  1291]
 [ 1304 24234]] Validation: score=0.9493   recall=0.9489   precision=0.9494

 [[27011  1415]
 [    3    52]] Test: score=0.9502   recall=0.9455   precision=0.0354

The most important features:  [13  3  7 28  9]


Important features

[[24971   669]
 [ 2115 23423]] Validation: score=0.9456   recall=0.9172   precision=0.9722

 [[27739   687]
 [    6    49]] Test: score=0.9757   recall=0.8909   precision=0.0666


In [35]:
list_dte0 = trees_and_forests(dt_e, smote);

Original features

[[25123   517]
 [ 2594 22944]] Validation: score=0.9392   recall=0.8984   precision=0.9780

 [[27901   525]
 [    8    47]] Test: score=0.9813   recall=0.8545   precision=0.0822

The most important features:  [13  3  7  9 28]


Important features

[[25269   371]
 [ 2753 22785]] Validation: score=0.9390   recall=0.8922   precision=0.9840

 [[28081   345]
 [    7    48]] Test: score=0.9876   recall=0.8727   precision=0.1221


In [36]:
list_rfcg0 = trees_and_forests(rfc_g, smote);

Original features

[[25517   123]
 [ 2742 22796]] Validation: score=0.9440   recall=0.8926   precision=0.9946

 [[28284   142]
 [    7    48]] Test: score=0.9948   recall=0.8727   precision=0.2526

The most important features:  [13 11 16  9  3  6  8 20 10  5  2]


Important features

[[25357   283]
 [ 2746 22792]] Validation: score=0.9408   recall=0.8925   precision=0.9877

 [[28155   271]
 [    5    50]] Test: score=0.9903   recall=0.9091   precision=0.1558


In [37]:
list_rfce0 = trees_and_forests(rfc_e, smote);

Original features

[[25561    79]
 [ 2949 22589]] Validation: score=0.9408   recall=0.8845   precision=0.9965

 [[28357    69]
 [    7    48]] Test: score=0.9973   recall=0.8727   precision=0.4103

The most important features:  [13 11  3  9 16  6  8 20 10  1  7  2]


Important features

[[25537   103]
 [ 2857 22681]] Validation: score=0.9422   recall=0.8881   precision=0.9955

 [[28329    97]
 [    8    47]] Test: score=0.9963   recall=0.8545   precision=0.3264


In [38]:
list_dtg1 = trees_and_forests(dt_g, smote1);

Original features

[[25283   357]
 [   57 25481]] Validation: score=0.9919   recall=0.9978   precision=0.9862

 [[28010   416]
 [    5    50]] Test: score=0.9852   recall=0.9091   precision=0.1073

The most important features:  [13 11  3  0 24 16]


Important features

[[25206   434]
 [   59 25479]] Validation: score=0.9904   recall=0.9977   precision=0.9833

 [[27945   481]
 [    4    51]] Test: score=0.9830   recall=0.9273   precision=0.0959


In [39]:
list_dte1 = trees_and_forests(dt_e, smote1);

Original features

[[25228   412]
 [   44 25494]] Validation: score=0.9911   recall=0.9983   precision=0.9841

 [[28022   404]
 [    4    51]] Test: score=0.9857   recall=0.9273   precision=0.1121

The most important features:  [13  3 11  9 10 28 25]


Important features

[[25233   407]
 [   64 25474]] Validation: score=0.9908   recall=0.9975   precision=0.9843

 [[28025   401]
 [    4    51]] Test: score=0.9858   recall=0.9273   precision=0.1128


In [40]:
list_rfcg1 = trees_and_forests(rfc_g, smote1);

Original features

[[25450   190]
 [  315 25223]] Validation: score=0.9901   recall=0.9877   precision=0.9925

 [[28220   206]
 [    8    47]] Test: score=0.9925   recall=0.8545   precision=0.1858

The most important features:  [13  3 16  2 11  9  6 18 15 20  0 17  1 10 23]


Important features

[[25447   193]
 [  503 25035]] Validation: score=0.9864   recall=0.9803   precision=0.9923

 [[28238   188]
 [    6    49]] Test: score=0.9932   recall=0.8909   precision=0.2068


In [41]:
list_rfce1 = trees_and_forests(rfc_e, smote1);

Original features

[[25467   173]
 [  235 25303]] Validation: score=0.9920   recall=0.9908   precision=0.9932

 [[28244   182]
 [    8    47]] Test: score=0.9933   recall=0.8545   precision=0.2052

The most important features:  [13  3  9  2 11 16 18  1 15  6 23 17  0]


Important features

[[25393   247]
 [  237 25301]] Validation: score=0.9905   recall=0.9907   precision=0.9903

 [[28138   288]
 [    7    48]] Test: score=0.9896   recall=0.8727   precision=0.1429


In [42]:
list_dtg2 = trees_and_forests(dt_g, smote2);

Original features

[[25069   470]
 [  330 25309]] Validation: score=0.9844   recall=0.9871   precision=0.9818

 [[27870   556]
 [    5    50]] Test: score=0.9803   recall=0.9091   precision=0.0825

The most important features:  [13  3 11 16  0]


Important features

[[24908   631]
 [  397 25242]] Validation: score=0.9799   recall=0.9845   precision=0.9756

 [[27745   681]
 [    5    50]] Test: score=0.9759   recall=0.9091   precision=0.0684


In [43]:
list_dte2 = trees_and_forests(dt_e, smote2);

Original features

[[24986   553]
 [  263 25376]] Validation: score=0.9841   recall=0.9897   precision=0.9787

 [[27770   656]
 [    4    51]] Test: score=0.9768   recall=0.9273   precision=0.0721

The most important features:  [13  3  9 16 11]


Important features

[[25243   296]
 [  830 24809]] Validation: score=0.9780   recall=0.9676   precision=0.9882

 [[28117   309]
 [    4    51]] Test: score=0.9890   recall=0.9273   precision=0.1417


In [44]:
list_rfcg2 = trees_and_forests(rfc_g, smote2);

Original features

[[25225   314]
 [  359 25280]] Validation: score=0.9868   recall=0.9860   precision=0.9877

 [[28073   353]
 [    6    49]] Test: score=0.9874   recall=0.8909   precision=0.1219

The most important features:  [13  3  9  2 11 16  1  6 18  0 20 23]


Important features

[[25143   396]
 [  360 25279]] Validation: score=0.9852   recall=0.9860   precision=0.9846

 [[27975   451]
 [    5    50]] Test: score=0.9840   recall=0.9091   precision=0.0998


In [45]:
list_rfce2 = trees_and_forests(rfc_e, smote2);

Original features

[[25201   338]
 [  388 25251]] Validation: score=0.9858   recall=0.9849   precision=0.9868

 [[28041   385]
 [    5    50]] Test: score=0.9863   recall=0.9091   precision=0.1149

The most important features:  [13  3  9 16  2 11 17  6 23 18  0  1 15  4 10]


Important features

[[25236   303]
 [  550 25089]] Validation: score=0.9833   recall=0.9785   precision=0.9881

 [[28093   333]
 [    6    49]] Test: score=0.9881   recall=0.8909   precision=0.1283


In [46]:
list_dtgada = trees_and_forests(dt_g, smoteada);

Original features

[[22552  3026]
 [ 1809 23791]] Validation: score=0.9055   recall=0.9293   precision=0.8872

 [[25134  3292]
 [    4    51]] Test: score=0.8843   recall=0.9273   precision=0.0153

The most important features:  [ 3 13 28  7  2  6 19  0  8]


Important features

[[22452  3126]
 [ 1839 23761]] Validation: score=0.9030   recall=0.9282   precision=0.8837

 [[25016  3410]
 [    4    51]] Test: score=0.8801   recall=0.9273   precision=0.0147


In [47]:
list_dteada = trees_and_forests(dt_e, smoteada);

Original features

[[23098  2480]
 [ 4730 20870]] Validation: score=0.8591   recall=0.8152   precision=0.8938

 [[25749  2677]
 [    5    50]] Test: score=0.9058   recall=0.9091   precision=0.0183

The most important features:  [13  3 11  7  0  9 25 18 16]


Important features

[[23089  2489]
 [ 4730 20870]] Validation: score=0.8589   recall=0.8152   precision=0.8934

 [[25739  2687]
 [    3    52]] Test: score=0.9056   recall=0.9455   precision=0.0190


In [48]:
list_rfcgada = trees_and_forests(rfc_g, smoteada);

Original features

[[24172  1406]
 [ 3196 22404]] Validation: score=0.9101   recall=0.8752   precision=0.9409

 [[26888  1538]
 [    1    54]] Test: score=0.9460   recall=0.9818   precision=0.0339

The most important features:  [13  3 11 16  9  6  2  0 19 18 20  4 17  7  1 28]


Important features

[[24004  1574]
 [ 2893 22707]] Validation: score=0.9127   recall=0.8870   precision=0.9352

 [[26702  1724]
 [    2    53]] Test: score=0.9394   recall=0.9636   precision=0.0298


In [49]:
list_rfceada = trees_and_forests(rfc_e, smoteada);

Original features

[[24253  1325]
 [ 3342 22258]] Validation: score=0.9088   recall=0.8695   precision=0.9438

 [[26964  1462]
 [    3    52]] Test: score=0.9486   recall=0.9455   precision=0.0343

The most important features:  [13  3 16 11  9  6 18  0  7 19 17 15 28 20  4  2 25 23]


Important features

[[23732  1846]
 [ 2988 22612]] Validation: score=0.9055   recall=0.8833   precision=0.9245

 [[26437  1989]
 [    2    53]] Test: score=0.9301   recall=0.9636   precision=0.0260


In [50]:
list_smote = np.hstack([list_dtg0, list_dte0, list_rfcg0, list_rfce0])
list_smote1 = np.hstack([list_dtg1, list_dte1, list_rfcg1, list_rfce1])
list_smote2 = np.hstack([list_dtg2, list_dte2, list_rfcg2, list_rfce2])
list_smoteada = np.hstack([list_dtgada, list_dteada, list_rfcgada, list_rfceada])

In [51]:
def good_feature(list_feat):
    values, counts = np.unique(list_feat, return_counts=True)
    max_occ = np.max(counts) # max number of occurence
    freq_feat = np.argwhere(counts==max_occ) # index of max_occ 
    features = np.ndarray.flatten(values[freq_feat])
    
    return features

# tbh, this method seems like it could potentially be overfitting;
## as the features that show up most often are selected,
### but a feature that shows up most often could have just barely shown up 

In [52]:
smote_features = good_feature(list_smote)
smote1_features = good_feature(list_smote1)
smote2_features = good_feature(list_smote2)
smoteada_features = good_feature(list_smoteada)

In [58]:
log_reg(lrcv, pd.DataFrame(X_ovrsamp)[smote_features], y_ovrsamp, pd.DataFrame(X_test[smote_features]), y_test)

-------------------------------------------
CV
-------------------------------------------
accuracy [0.93248401 0.93242539 0.93261991]
precision [0.97070246 0.97100586 0.97011123]
recall [0.89191759 0.89150083 0.89277574]
f1 [0.92964381 0.92955643 0.92983824]
roc_auc [0.97887998 0.97911586 0.97917339]
-------------------------------------------
Overall
-------------------------------------------
[[24928   712]
 [ 2760 22778]]
Validation - Recall: 0.8919 Precision: 0.9697 Accuracy: 0.9322
[[27696   730]
 [    4    51]]
Test - Recall: 0.9273 Precision: 0.0653 Accuracy: 0.9742


In [59]:
log_reg(lrcv_1, pd.DataFrame(X_ovrsamp1)[smote1_features], y_ovrsamp1, pd.DataFrame(X_test)[smote1_features], y_test)

-------------------------------------------
CV
-------------------------------------------
accuracy [0.9725989  0.97354983 0.97219472]
precision [0.97855702 0.97973481 0.97866554]
recall [0.96638623 0.96711554 0.96544808]
f1 [0.97243354 0.97338428 0.97201188]
roc_auc [0.99691078 0.99717831 0.99681731]
-------------------------------------------
Overall
-------------------------------------------
[[25064   576]
 [  907 24631]]
Validation - Recall: 0.9645 Precision: 0.9771 Accuracy: 0.9710
[[27832   594]
 [    4    51]]
Test - Recall: 0.9273 Precision: 0.0791 Accuracy: 0.9790


In [60]:
log_reg(lrcv_2, pd.DataFrame(X_ovrsamp2)[smote2_features], y_ovrsamp2, pd.DataFrame(X_test)[smote2_features], y_test)

-------------------------------------------
CV
-------------------------------------------
accuracy [0.97046257 0.96999342 0.96964802]
precision [0.97248139 0.97326126 0.97244832]
recall [0.9683127  0.96652769 0.96667058]
f1 [0.97039257 0.96988279 0.96955084]
roc_auc [0.99500219 0.99518638 0.9949349 ]
-------------------------------------------
Overall
-------------------------------------------
[[24831   708]
 [  827 24812]]
Validation - Recall: 0.9677 Precision: 0.9723 Accuracy: 0.9700
[[27657   769]
 [    4    51]]
Test - Recall: 0.9273 Precision: 0.0622 Accuracy: 0.9729


In [61]:
log_reg(lrcv_ada, pd.DataFrame(X_ovrsampada)[smoteada_features], y_ovrsampada, pd.DataFrame(X_test)[smoteada_features], y_test)

-------------------------------------------
CV
-------------------------------------------
accuracy [0.83442212 0.83303373 0.83249204]
precision [0.86304625 0.86184201 0.86005502]
recall [0.79498202 0.79321036 0.79419773]
f1 [0.82761707 0.82610318 0.82581545]
roc_auc [0.91686074 0.91655789 0.91588142]
-------------------------------------------
Overall
-------------------------------------------
[[22285  3293]
 [ 5141 20459]]
Validation - Recall: 0.7992 Precision: 0.8614 Accuracy: 0.8352
[[24897  3529]
 [    1    54]]
Test - Recall: 0.9818 Precision: 0.0151 Accuracy: 0.8761


In [ ]:
stop  here

In [ ]:
# inverse balance

## initial results showed promise, however
## intuitively, what is the sense of doing this?
### in the regular imbalance, there was heavy bias towards the majority class
### would not reversing the imbalance create bias towards the "minority class"
#### there are already other methods of doing some form of this
##### i.e., using predict_proba, lowering threshold for classification
#### is there evidence/reasoning reversed imbalance does not work or is not used?
##### bit unorthodox to say the least, or maybe it isn't

# HOWEVER, must test for each combination of undersampling/oversampling method
## may just use ClusterCentroid and ADASYN
### have to extract PCs first

# AND how to determine what ratio to use?

In [ ]:
# useless(?) stuff below

In [ ]:
def inverse_balance(X_Train, y_Train, underSamp_model, ovrSamp_model, underSamp_ratio=0.005):
   
    ratio1 = np.bincount(y_Train)[1]
    ratio0 = (ratio1 / underSamp_ratio) - ratio1 # underSamp_ratio arbitrary; 
    # original ratio = 0.0017
    X_unsamp, y_unsamp = underSamp_model(random_state=0, ratio={0:ratio0, 1:ratio1}).fit_sample(X_Train, y_Train)
    X_ovrsamp, y_ovrsamp = ovrSamp_model[0], ovrSamp_model[1]
    
    
    df_ovr = pd.DataFrame(X_ovrsamp)
    df_ovr['Class'] = y_ovrsamp

    df_under = pd.DataFrame(X_unsamp)
    df_under['Class'] = y_unsamp

    df_ovr1 = df_ovr[df_ovr.Class==1]
    df_under0 = df_under[df_under.Class==0]
    
    df_invrs_bal = pd.concat([df_ovr1, df_under0]).reset_index()
    inv_bal_index = df_invrs_bal['index'] # in case index is needed
    
    df = df_invrs_bal.drop('index', 1) 
    
    
    X_train = df.iloc[:, :-1] # column '28' is 'Amount' 
    y_train = df['Class'].copy()
    
    
    lrcv = LogisticRegressionCV(Cs=c_params, random_state=777, scoring='recall')
    lrcv.fit(X_train, y_train)
    y_ = lrcv.predict(X_test) # can call global variables?
    
    print('Accuracy %0.4f'% lrcv.score(X_test, y_test), '\nConfusion Matrix %0.4'% confusion_matrix(y_test, y_), 
          'Recall %0.4f'% recall_score(y_test, y_))

In [ ]:
from imblearn.under_sampling import NearMiss, RandomUnderSampler, ClusterCentroids

In [ ]:
X_unsamp, y_unsamp = RandomUnderSampler(random_state=0, ratio={0:86167, 1:433}).fit_sample(X_Train, y_Train)

In [ ]:
print(np.bincount(y_Train), np.bincount(y_unsamp))

In [ ]:
# combine y_ovrsamp where 'Class' = 1 with
# with y_unsamp where 'Class' = 0
## must join to respective X_matrix first

In [ ]:
df_ovr = pd.DataFrame(X_ovrsamp)
df_ovr['Class'] = y_ovrsamp

In [ ]:
df_ovr = pd.DataFrame(X_ovrsamp)
df_ovr['Class'] = y_ovrsamp

df_under = pd.DataFrame(X_unsamp)
df_under['Class'] = y_unsamp

df_ovr1 = df_ovr[df_ovr.Class==1]
df_under0 = df_under[df_under.Class==0]

In [ ]:
df_ovr1 = df_ovr[df_ovr.Class==1]
df_under0 = df_under[df_under.Class==0]

In [ ]:
print (np.bincount(df_under0.Class), np.bincount(df_ovr1.Class) ) 
# reversed the imbalance

In [ ]:
df_invrs_bal = pd.concat([df_ovr1, df_under0]).reset_index()
inv_bal_index = df_invrs_bal['index'] # in case index is needed
df0 = df_invrs_bal.drop('index', 1) 

In [ ]:
X0 = df0.iloc[:, :-1] # column '28' is 'Amount' 
y0 = df0['Class'].copy()

In [ ]:
lrcv0 = LogisticRegressionCV(Cs=c_params, random_state=777, scoring='recall')

In [ ]:
lrcv0.fit(X0, y0)

In [ ]:
lrcv0.score(X_test, y_test)

In [ ]:
y_0 = lrcv0.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_0) 

In [ ]:
recall_score(y_test, y_0)

In [ ]:
# check y_test index for relationship with errors 

In [ ]:
lrcv.fit(X_ovrsamp, y_ovrsamp)
lrcv_1.fit(X_ovrsamp1, y_ovrsamp1)
lrcv_2.fit(X_ovrsamp2, y_ovrsamp2)
lrcv_ada.fit(X_ovrsampada, y_ovrsampada)
lrcv_rand.fit(X_ovrsamprand, y_ovrsamprand)

In [ ]:
# done prior to addition of X_val, y_val
# keep test or use val?

y_ada = lrcv_ada.predict(X_test)
y_0 = lrcv.predict(X_test) # regular smote
y_1 = lrcv_1.predict(X_test) # smote borderline 1
y_2 = lrcv_2.predict(X_test) 
y_rand = lrcv_rand.predict(X_test) 

In [ ]:
indices = []
Y = pd.DataFrame(y_test, columns=['Class'])
for y_ in [y_ada, y_0, y_1, y_2, y_rand]:
    fn = Y[y_ != y_test] # all incorrect predictions
    type2 = fn[fn['Class']==1].index.values # only type 2 
    indices.append(type2)

In [ ]:
indices

In [ ]:
a, b, c, d, e = indices # there isn't too much difference in type 2 errors

np.in1d(a,b).sum() # other sampling methods provide similar methods


In [ ]:
# names = ['smote', 'smote_borderline1', 'smote_bordeline2', 'ADASYN', 'Random_over_samp']
# clfs = [lrcv, lrcv_1, lrcv_2, lrcv_ada, lrcv_rand]
# samps = [smote, smote1, smote2, smoteada, smoterand]

# for name, model, samp in zip(names, clfs, samps):
#     model.fit(samp[0], samp[1]) # samp[0] == X; samp[1] == y for each 
#     y_ = model.predict(X)
#     mat = confusion_matrix(y, y_)
#     print('Entire set\n')
#     print('\t', name)
#     print(mat)
#     print('\t\tRecall: %0.4f'% recall_score(y,y_), 
#           'Precision: %0.4f'% precision_score(y,y_),
#           'Accuracy: %0.4f\n'% model.score(X,y))

In [ ]:
# names = ['smote', 'smote_borderline1', 'smote_bordeline2', 'ADASYN', 'Random_over_samp']
# clfs = [lrcv, lrcv_1, lrcv_2, lrcv_ada, lrcv_rand]
# samps = [smote, smote1, smote2, smoteada, smoterand]

# for name, model, samp in zip(names, clfs, samps):
#     model.fit(samp[0], samp[1]) # samp[0] == X; samp[1] == y for each 
#     y_ = model.predict(X_test)
#     mat = confusion_matrix(y_test, y_)
#     print('Test Set\n')
#     print('\t', name)
#     print(mat)
#     print('\t\tRecall: %0.4f'% recall_score(y_test,y_), 
#           'Precision: %0.4f'% precision_score(y_test,y_),
#           'Accuracy: %0.4f\n'% model.score(X_test,y_test))

In [ ]:
# useless stuff above

In [ ]:
# linear regression assumptions with uniform distribution
## convert uniform to normal
### bin mapping, has to be equal range?

# pca transform in order of explained_variance_?

# trees, class_weight parameter corrects imbalance?